In [1]:
#!transformers-cli login
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!sudo apt-get install git-lfs
!git lfs install

'sudo' is not recognized as an internal or external command,
operable program or batch file.
'sudo' is not recognized as an internal or external command,
operable program or batch file.


Updated git hooks.
Git LFS initialized.


In [3]:
from transformers import (
    AutoConfig,
    AutoModelForSeq2SeqLM,
    AutoTokenizer
)

In [11]:
model_path = "../model/bart-large-samsum"
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [9]:
#model.push_to_hub("bart-large-samsum")
tokenizer.push_to_hub("bart-large-samsum")

In [15]:
import json
# read eval and test results 
with open(f"{model_path}/eval_results.json") as f:
    eval_results_raw = json.load(f)
    eval_results={}
    eval_results["eval_rouge1"] = eval_results_raw["eval_rouge1"]
    eval_results["eval_rouge2"] = eval_results_raw["eval_rouge2"]
    eval_results["eval_rougeL"] = eval_results_raw["eval_rougeL"]
    eval_results["eval_rougeLsum"] = eval_results_raw["eval_rougeLsum"]

with open(f"{model_path}/predict_results.json") as f:
    test_results_raw = json.load(f)
    test_results={}
    test_results["predict_rouge1"] = test_results_raw["predict_rouge1"]
    test_results["predict_rouge2"] = test_results_raw["predict_rouge2"]
    test_results["predict_rougeL"] = test_results_raw["predict_rougeL"]
    test_results["predict_rougeLsum"] = test_results_raw["predict_rougeLsum"]

In [23]:
MODEL_CARD_TEMPLATE = """
---
language: en
tags:
- azureml
- bart
- summarization
license: apache-2.0
datasets:
- samsum
model-index:
- name: {model_name}
  results:
  - task: 
      name: Abstractive Text Summarization
      type: abstractive-text-summarization
    dataset:
      name: "SAMSum Corpus: A Human-annotated Dialogue Dataset for Abstractive Summarization" 
      type: samsum
widget:
- text: | 
    Henry: Hey, is Nate coming over to watch the movie tonight?
    Kevin: Yea, he said he'll start heading over here at around 7. Have you taken out the garbage yet?
    Henry: Oh I forgot. I'll do that once I'm finished with my assignment for my math class.
    Kevin: Yea, you should take it out as soon as possible. Also, Nate is bringing his cat.
    Henry: Okay, see you then.
---

## `{model_name}`

This model was trained using AzureML.


## Usage
```python
from transformers import pipeline
summarizer = pipeline("summarization", model="henryu-lin/{model_name}")

conversation = '''Henry: Hey, is Nate coming over to watch the movie tonight?
Kevin: Yea, he said he'll start heading over here at around 7. Have you taken out the garbage yet?
Henry: Oh I forgot. I'll do that once I'm finished with my assignment for my math class.
Kevin: Yea, you should take it out as soon as possible. Also, Nate is bringing his cat.
Henry: Okay, see you then.
'''
nlp(conversation)
```

## Results

| key | value |
| --- | ----- |
{eval_table}
{test_table}



"""

# Generate model card (todo: add more data from Trainer)
model_card = MODEL_CARD_TEMPLATE.format(
    model_name=f"bart-large-samsum",
    #hyperparameters=json.dumps(hyperparameters, indent=4, sort_keys=True),
    eval_table="\n".join(f"| {k} | {v} |" for k, v in eval_results.items()),
    test_table="\n".join(f"| {k} | {v} |" for k, v in test_results.items()),
)

with open(f"{model_path}/README.md", "w") as f:
    f.write(model_card)

In [7]:
model_name = "facebook/bart-large"
xsum_config = AutoConfig.from_pretrained("./bart-large-xsum.json", cache_dir="./cache")
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir="./cache")
model = AutoModelForSeq2SeqLM.from_pretrained(model_name, config=xsum_config, cache_dir="./cache")

In [8]:
model.save_pretrained("../model/bart-large")
tokenizer.save_pretrained("../model/bart-large")

('../model/bart-large/tokenizer_config.json',
 '../model/bart-large/special_tokens_map.json',
 '../model/bart-large/vocab.json',
 '../model/bart-large/merges.txt',
 '../model/bart-large/added_tokens.json',
 '../model/bart-large/tokenizer.json')